<a href="https://colab.research.google.com/github/schase15/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Steven_Chase_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [386]:
# Look at dataset
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [389]:
# Look at cleaned dataframe
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [390]:
# Subset data for BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS'
# SALE_PRICE is more than $100,000 and less than $2 million

# Create conditions
condition1= df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'
condition2= [(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)]

# Pass in conditions
df= df[condition1]
df= df[condition2[0]]

# Look at subset of data
print(df.shape)
df.head()

(3151, 21)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [391]:
# Look at easement and apartment number, there are a lot of NaN
print(df['EASE-MENT'].isnull().sum())
print(df['APARTMENT_NUMBER'].isnull().sum())

3151
3150


In [392]:
# Drop easement, they are all NaN
df= df.drop('EASE-MENT', axis=1)

# Drop apartment number, there is only 1 without NaN
df= df.drop('APARTMENT_NUMBER', axis= 1)

df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [0]:
# LAND_SQUARE_FEET is a string but should be an integer
# Typical methods weren't working, had to manually remove commas
# for loop to go through column and remove commas
# Resave over LAND_SQUARE_FEET

df['LAND_SQUARE_FEET']= [int(comma.replace(',', '')) for comma in df['LAND_SQUARE_FEET']]

In [394]:
# Split train and test
# Train Jan- March 2009
# Test April 2009
train = df[df['SALE_DATE']< '04/01/2019']
test = df[df['SALE_DATE'] >= '04/01/2019']

# Look at value counts of each to make sure filtering worked
print('Train: ', train['SALE_DATE'].value_counts().index)
print('Test: ', test['SALE_DATE'].value_counts().index)

Train:  Index(['01/31/2019', '03/29/2019', '02/28/2019', '01/15/2019', '01/24/2019',
       '01/30/2019', '01/25/2019', '01/17/2019', '02/15/2019', '02/22/2019',
       '03/28/2019', '01/18/2019', '01/11/2019', '01/29/2019', '02/08/2019',
       '01/22/2019', '03/22/2019', '02/12/2019', '03/08/2019', '03/15/2019',
       '03/11/2019', '01/16/2019', '02/01/2019', '03/26/2019', '02/20/2019',
       '01/28/2019', '01/23/2019', '03/14/2019', '01/10/2019', '03/12/2019',
       '01/03/2019', '02/14/2019', '03/25/2019', '02/07/2019', '03/01/2019',
       '02/21/2019', '01/09/2019', '02/27/2019', '02/25/2019', '02/26/2019',
       '03/19/2019', '02/19/2019', '01/08/2019', '03/07/2019', '03/27/2019',
       '01/04/2019', '02/05/2019', '02/13/2019', '03/21/2019', '01/07/2019',
       '03/06/2019', '03/04/2019', '02/11/2019', '03/18/2019', '01/14/2019',
       '02/04/2019', '03/20/2019', '03/13/2019', '03/05/2019', '01/02/2019',
       '02/06/2019', '01/21/2019', '02/18/2019', '01/01/2019', '02/1

In [395]:
# Look at categorical columns and see which have high cardinality 
# anything that has more than 50 unique values

train.describe(exclude= 'number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,2507,2507
unique,5,6,1,2,13,2497,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,294 FREEBORN STREET,A1,01/31/2019
freq,1204,2382,2507,2476,919,2,919,78


In [0]:
# High cardinality that we want to drop
high_cardinality= ['ADDRESS', 'SALE_DATE']

# Define target and features
target = ['SALE_PRICE']
features= train.columns.drop(target + high_cardinality)

# Split test and train data into features and target
X_train= train[features] 
y_train= train[target]

X_test= test[features]
y_test= test[target]

In [0]:
# One hot encode features

# Import and define encoder
import category_encoders as ce
encoder= ce.OneHotEncoder(use_cat_names= True)

# Fit and transform features in training data
X_train= encoder.fit_transform(X_train)

# Transform features in testing data
X_test= encoder.transform(X_test)

In [398]:
# Look at dataset
X_test.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
18235,0,0,1,0,0,1,0,0,0,0,0,1,1,0,5913,878,0,1,0,0,0,0,0,0,0,0,0,0,0,10471.0,1.0,0.0,1.0,5000,2272.0,1930.0,1,0,1,0,0,0,0,0,0,0,0,0
18239,0,0,1,0,0,1,0,0,0,0,0,1,1,0,5488,48,0,0,0,0,1,0,0,0,0,0,0,0,0,10465.0,1.0,0.0,1.0,2500,720.0,1935.0,1,0,0,0,0,1,0,0,0,0,0,0
18244,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5936,31,0,1,0,0,0,0,0,0,0,0,0,0,0,11209.0,1.0,0.0,1.0,2880,2210.0,1925.0,1,0,1,0,0,0,0,0,0,0,0,0
18280,1,0,0,0,0,1,0,0,0,0,0,1,1,0,7813,24,0,0,1,0,0,0,0,0,0,0,0,0,0,11210.0,1.0,0.0,1.0,1305,1520.0,1915.0,1,0,0,1,0,0,0,0,0,0,0,0
18285,1,0,0,0,0,1,0,0,0,0,0,1,1,0,8831,160,1,0,0,0,0,0,0,0,0,0,0,0,0,11229.0,1.0,0.0,1.0,1800,840.0,1925.0,1,1,0,0,0,0,0,0,0,0,0,0


In [399]:
# Feature selection with SelectKBest

# Import and define selector, use 15 best categories
from sklearn.feature_selection import SelectKBest, f_regression
selector= SelectKBest(score_func= f_regression, k=15)

# Use selector to fit and transform train data
X_train_selected= selector.fit_transform(X_train, y_train)

# Use selector to transform test data
X_test_selected= selector.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/sci

In [400]:
# Look at selected features

# Pull out selected fetures - true or false across column headers
selected= selector.get_support()

# Create a list of all columns
all_names= X_train.columns

# Subset of column names that were selected (True from first step)
selected_names= all_names[selected]
selected_names.tolist()

['BOROUGH_3',
 'BOROUGH_2',
 'BOROUGH_5',
 'NEIGHBORHOOD_OTHER',
 'NEIGHBORHOOD_FLUSHING-NORTH',
 'NEIGHBORHOOD_FOREST HILLS',
 'BLOCK',
 'BUILDING_CLASS_AT_PRESENT_A5',
 'BUILDING_CLASS_AT_PRESENT_A3',
 'ZIP_CODE',
 'COMMERCIAL_UNITS',
 'TOTAL_UNITS',
 'LAND_SQUARE_FEET',
 'GROSS_SQUARE_FEET',
 'BUILDING_CLASS_AT_TIME_OF_SALE_A3']

In [401]:
# Ridge regression model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score

for alpha in [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]:

# Define model
    model= Ridge(alpha=alpha, normalize= True)
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

# Test MAE
    mae= mean_absolute_error(y_test, y_pred)
    print('Ridge regression with alpha of', alpha, ', MAE is: ', mae)

Ridge regression with alpha of 0.001 , MAE is:  155928.80601411837
Ridge regression with alpha of 0.01 , MAE is:  155629.9898094683
Ridge regression with alpha of 0.1 , MAE is:  155470.17359729038
Ridge regression with alpha of 1.0 , MAE is:  168231.09304575986
Ridge regression with alpha of 10.0 , MAE is:  199218.29454297133
Ridge regression with alpha of 100.0 , MAE is:  209945.11094697297


In [402]:
# Try it with values closer to 0.1 which was the best alpha last time
import numpy as np

for alpha in np.arange(0.01, 0.1, 0.01):

# Define model
    model= Ridge(alpha=alpha, normalize= True)
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

# Test MAE
    mae= mean_absolute_error(y_test, y_pred)
    print('Ridge regression with alpha of', alpha, ', MAE is: ', mae)

Ridge regression with alpha of 0.01 , MAE is:  155629.9898094683
Ridge regression with alpha of 0.02 , MAE is:  155479.41674161618
Ridge regression with alpha of 0.03 , MAE is:  155386.6736283119
Ridge regression with alpha of 0.04 , MAE is:  155326.50640554927
Ridge regression with alpha of 0.05 , MAE is:  155293.72246476132
Ridge regression with alpha of 0.060000000000000005 , MAE is:  155282.8022371059
Ridge regression with alpha of 0.06999999999999999 , MAE is:  155298.91300311656
Ridge regression with alpha of 0.08 , MAE is:  155345.68284691102
Ridge regression with alpha of 0.09 , MAE is:  155401.94693407667


In [403]:
# Alpha value of 0.06 gives the lowest MAE

# Set alpha
alpha= 0.06

# Define and fit model
model= Ridge(alpha=alpha, normalize= True)
model.fit(X_train, y_train)

# Make predictions
y_pred= model.predict(X_test)

# Test MAE
mae= mean_absolute_error(y_test, y_pred)
print('Ridge regression with alpha of', alpha, ', MAE is: ', mae)

# R2 test value
r2_test= r2_score(y_test, y_pred)
print('R^2 value: ', r2_test)

Ridge regression with alpha of 0.06 , MAE is:  155282.8022371059
R^2 value:  0.4210479538970584


In [404]:
# Baseline to compare with 

guess= y_train.mean()
y_pred= [guess] * len(y_test)
mae= mean_absolute_error(y_test, y_pred)
mae

211564.20053849227

In [0]:
### STRETCH GOALS

In [406]:
# Linear regression
from sklearn.linear_model import LinearRegression
model= LinearRegression()

# Already defined test, train, target and features above

# Fit the model
model.fit(X_train, y_train)

# Make predicitons
y_pred= model.predict(X_test)

# Calculate the error
mae= mean_absolute_error(y_test, y_pred)
mae

155944.19177018633

In [407]:
# RidgeCV
from sklearn.linear_model import RidgeCV

# Define alphas
alphas = [0.01, 0.06, 0.1, 1.0, 10.0, 100.0]

# Define and fit model
ridge= RidgeCV(alphas=alphas, normalize=True)
ridge.fit(X_train, y_train)

# Make predictions
y_pred= ridge.predict(X_test)

# Calculate error
mae= mean_absolute_error(y_test, y_pred)
print(mae)

# Show what alpha was used
ridge.alpha_


155629.98980946562


0.01